> Se importan las librerías a utilizar

In [0]:
import numpy as np

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
from pyspark.sql.functions import col

from pyspark.sql.types import StringType,DoubleType
import pyspark.sql.functions as F
from pyspark.sql.types import *

spark =SparkSession.builder.appName("mi_app").getOrCreate()
from decimal import *
sc=spark.sparkContext

from pyspark.sql.functions import lit
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer,OneHotEncoder, VectorAssembler
from pyspark.sql.functions import col
import csv
import numpy as np
import pandas as pd
import gzip
from datetime import date
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_auc_score
from numpy import arange
from matplotlib import pyplot
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pyspark.ml.feature import OneHotEncoder
from pyspark.sql.functions import col

from pyspark.sql.types import StringType,DoubleType
from pyspark.sql.types import (StringType, BooleanType, IntegerType, FloatType, DateType)
  
from pyspark.sql.functions import col,isnan, when, count
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql import Window


#Librerias de Spark que se utilizan en la generacion del modelo de regresion lineal 
from pyspark.sql.functions import corr


from pyspark.ml.regression import LinearRegression



from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import RegressionEvaluator, BinaryClassificationEvaluator

In [0]:
%sh
pip install mlflow
/databricks/python_shell/scripts/PythonShell.py configure

> Se importan las 3 bases que fueron el resultado del proceso de preparación de los datos:

> Se hace un filtrado para determinadas columnas

> Se categorizan valores numéricos

In [0]:
#raw_datax = spark.read.options(inferSchema='True',header='True')('/dbfs/gold/icbf-gai/BaseModeloProbabilisticoSabana')



path_to_data="dbfs:/dbfs/gold"
raw_datax = spark.read.options(inferSchema='True',header='True').format("delta").load(path_to_data)


In [0]:

def tiposendf(raw_datax):# funcion que da los tipos en el df en una lista 
    tipos=raw_datax.dtypes
    ele=[]
    for i in range(len(tipos)):
        ele.append(tipos[i][1])
    uniq = []
    for x in ele:
        if x not in tipo:
            uniq.append(x)
            tipo.add(x)
    return(uniq)
  
def filtrotype(df, tipo):# funcion que da las columnas de cierto tipo en especifico 
    my_list = []
    for i in df.dtypes:
        if i[1].startswith(tipo):
            my_list.append(i)
    return my_list

def filtro(df, ini,fin): # filtro al cual entra como parametro un df y 2 palabras claves de inicio y fin del nombre de las columnas y devuelve un df que no tuene dichas columnas 
    lista=df.columns
    my_list = []
    for i in lista:
        if i.startswith(ini) and i.endswith(fin):
             my_list.append(i)
        
    df = df.drop(*my_list)
    return df

def filtro1(df, key,idi): # funcion  a la cual entra un df y dos palabras clves entre comillas y devuelve la lista de las columnas que tienen esas palabras 
    lista=df.columns
    arr = np.array(lista)
    my_list = []
    for i in arr:
        #for j in len(lista[i]):
        if key in i:
            my_list.append(i)
                
        if idi in i:
            my_list.append(i)
    return my_list

 ## fun repetidas 

def filtro2(df, key): # filtro al cual entra como parametro un df y  una palabra clave en comillas y devuelve una lista con las columnas del df que contienen esa palabra clave 
    lista=df.columns
    arr = np.array(lista)
    my_list = []
    for i in arr:
        #for j in len(lista[i]):
        if key in i:
            my_list.append(i)
                
    return my_list

def filtrofin(df, fin): # Filtro al cual entra un df y una palabra clave y  devuelve una lista con el nombre de todas las columnas que terminen con dichas ´palabras clave 
    lista=df.columns
    my_list = []
    for i in lista:
        if  i.endswith(fin):
             my_list.append(i)

    return my_list

In [0]:
len(raw_datax.columns)

# limpieza 

## se utilizan los filtros para eliminar columnas repetidas o que no aportan informacion relevante, como identificacion por nombres, direcciones, fechas o columnas que ya estan modeladas por otras columnas en el df 

se eliminan 210 columnas con dichos filtros

In [0]:
nom=filtro1(raw_datax, 'nom','Nom')
idi=filtro2(raw_datax, 'IdBeneficiarioT')
ins =filtro2(raw_datax, 'INS')
Dir=filtro1(raw_datax, 'Dir','dir')
fecha=filtro1(raw_datax, 'Fecha','fecha')
flag=filtro1(raw_datax, 'Flag','PresentaCarneVacunacion')
NoTomaT=filtro1(raw_datax, 'Flag','NoTomaT')
TomaT=filtro2(raw_datax, 'TomaT')
raw_datax=raw_datax.drop(*nom)
raw_datax=raw_datax.drop(*idi)
raw_datax=raw_datax.drop(*Dir)
raw_datax=raw_datax.drop(*fecha)
raw_datax=raw_datax.drop(*flag)
raw_datax=raw_datax.drop(*NoTomaT)
raw_datax=raw_datax.drop(*TomaT)
raw_datax=raw_datax.drop(*ins)

listeli=['depmuni','depto','munic','zona','sector','seccion','manzana','comuna','barrio','vereda','NOM_DPTO','NOM_MPIO','NumeroDocumento',
'SegundoApellido','PrimerApellido','NumeroDocumento','NoTomaT','ficha','teles','ape1','ape2','documen','ind_estudia','tip_origen_agua','ind_tiene_alcantarillado','tip_mat_paredes','tip_mat_pisos','ind_tiene_acueducto','num_cuartos_dormir','tip_sanitario','ide_ficha_origen','num_ficha', 'Jefe_UG',  'seg_nom_informante',
'tip_cuidado_niños',
'ide_firma_informante',
'Email_contacto',
'Cod_dpto',
'Cod_mpio',
'coord_x_manual_rec',
'coord_y_manual_rec',
'coord_x_auto_rec',
'coord_y_auto_rec',
'Dir_Chip',
'ide_persona',
'ind_escaner',
'pri_apellido',
'seg_apellido',
'pri_nombre',
'seg_nombre',
'sexo_persona',
'tip_documento',
'num_documento',
'fec_nacimiento',
'edad_calculada',
'fec_documento',
'cod_mpio_documento',
'cod_dpto_documento',
'cod_pais_documento',
'ind_discap_ver',
'ind_discap_oir',
'ind_discap_hablar',
'ind_discap_moverse',
'ind_discap_bañarse',
'ind_discap_salir',
'ind_discap_entender',
'ind_discap_ninguna',
'ind_fondo_pensiones',
'tip_actividad_mes',
'num_sem_buscando',
'tip_empleado',
'Ide_hogar',
'ide_informante',
'pri_nom_informante',
'seg_nom_informante',
'pri_ape_informante',
'seg_nom_informante',
'ide_firma_informante',
'Email_contacto',
'num_tel_contacto',
'Grupo',
'Nivel',
'fec_actualizacion_cns',
'tip_mat_paredes',
'tip_mat_pisos']
raw_datax=raw_datax.drop(*listeli)
#ZonaUbicacionBeneficiario

# funciones para limpiar y crear las sabanas de datos

In [0]:
def nulls(df):# funcion que da el procentaje de nulls por columna de un df de entrada
    cols=df.columns
    rows=df.count()
    df=df.withColumn('numNulls1',sum(df[col].isNull().cast('int') for col in cols))
    Dict_Null = {col:df.filter(df[col].isNull()).count()/rows for col in cols}# crea un diccionario con
    #el nombre de la columna y la cantidad de nulls en    dicha columna
    val=Dict_Null.values() 
    key=Dict_Null.keys() 
    key1=list(key)#lista con nombre de la columna
    filt=[]
    val1=list(val)
    return(Dict_Null)
    # crea un diccionariocon el nombre de la columna y la cantidad de nulls en dicha columna
    

In [0]:
# la funcion filtro 4 elimina las columnas asociadas a las tomas no deseadas se debe especificar el # de tomas que se quieren tener en cuenta, 
#luego elimina las columnas que tienen un porcentaje superior de nulls al deseado el cual entra por parametro 
#devuleve un df que segun un porcentaje de nulls predefinido acota la cantidad de columnas 
def filtro4(df,Notomas,porcentaje):
    df=df.select([when(col(c)=="","Null").otherwise(col(c)).alias(c) for c in df.columns]) #llena las columnas con null 
    p=df.select(filtro1(df, 'EstadoPesoTalla', 'IdBeneficiario')) # selecciona solo las columnas EstadoPesoTalla
    p1=p.drop(*filtro2(df, 'IdBeneficiarioT')) #dataframe sin las columnas  IdBeneficiarioT
    listaEPT=p1.columns # lista con las columnas EPT+ Idbeneficiario
    p1=p1.withColumn('numNulls', sum(p1[col].isNull().cast('int') for col in listaEPT)) #crea la columna numNulls que indica la cantidad de nulls en las  columnas EstadoPesoTalla. p1 un data frame con las columnas estado peso el id y la columna numnulls
    f=len(listaEPT) # cantidad de tomas mas el id del ben sin um nulls
    #h=(18-Notomas)
    h=(f-Notomas+2)# esto!
    p1 =p1.filter(p1.numNulls== h) # filtra las columnas segun el no de tomas deseado
    listaEPTf=p1.columns # cantidad de tomas mas el id del ben sin columna num nulls
    
    lista3=listaEPTf[Notomas:]# lista con las columnas no deseadas 
    
    p1=p1.drop(*lista3) # dataframe con las columnas de las EPT deseadas
    colfin=p1.columns
    
    lista4=p1.select('IdBeneficiario').rdd.map(lambda x : x[0]).collect()# hace una lista con los usuarios que cumplen la condicion de tener las tomas 
    #parte2
    
    df1=df.filter(df.IdBeneficiario.isin(lista4)) # elimina las filas de los usuarios que no cumplen en el df de entrada
    #lista5=df1.columns #columnas finale 
    #df1=df1.withColumn('numNulls1', sum(df1[col].isNull().cast('int') for col in lista5)) # crea una columna contando los nulls por fila 
    df1=df1.drop(*lista3)
    r=df1.count()
    c=df1.columns
    Dict_Null = {col:df1.filter(df1[col].isNull()).count() for col in c}# crea un diccionariocon el nombre de la columna y la cantidad de nulls en dicha columna
    val=Dict_Null.values() 
    key=Dict_Null.keys() 
    key1=list(key)#lista con nombre de la columna
    filt=[]
    val1=list(val) # lista con la cantidad de nulls
    for i in range(len(val1)):# calculo del porcentaje de nulls en dicha columna comparado con el numero total de filas 
        if val1[i]/r > porcentaje:# mayor al porcentaje se elimina columna
            filt.append(key1[i])# lista con las columnas que tienen un porcentaje de nulls mayor al entrado por parametro
    #print(filt)
    df2=df1.drop(*filt)# elimina dichas columnas 
    #hasta aca esta bien
    
    notomas=(range(1,Notomas+2,1))
    lista6=[]
    for c in notomas: # crea una lista con el nombre de las columnas de las tomas que no se consideran 
        name='T'
        if c in lista3:
            #lista6.append(filtrofin(df2, "%s%s"%(name,c))
            lista6.extend(filtrofin(df2, "%s%s"%(name,c) ))
    df2=df2.drop(*lista6) # elimina las columnas relacionadas con las tomas que no se tienen en cuenta 
    return df2

In [0]:

def isInt(num):
    try:
        int(num)
        return True
    except ValueError or TypeError:
        return False
    
def isfloat(num):
    try:
        float(num)
        return True
    except ValueError or TypeError:
        return False

def catInt(df): # esta funcion  evalua todas las columnas del df y determina si es int u la guarda en una lsta 
    f=df.first()
    c=df.columns
    inte=[]
    for i in range(0,len(f),1):
        if isinstance(f[i], int)==True and type(f[i])== int:
            inte.append(c[i])
    return(inte)

def catFlo(df): # esta funcion  evalua todas las columnas del df y determina si es float u la guarda en una lsta 
    f=df.first()
    c=df.columns
    #fp=f.topandas()
    flo=[]
    inte=catInt(df)
    for i in range(0,len(f),1):
        if isinstance(f[i], float)==True and type(f[i])== decimal.Decimal :
            flo.append(c[i])
            
    return(flo)
def catstring(df): # esta funcion  evalua todas las columnas del df y determina si es String u la guarda en una lsta 
    f=df.first()
    c=df.columns
    st=[]
    for i in range(0,len(f),1):
        if isinstance(f[i], str)==True and type(f[i])!=int and type(f[i])!=type(Decimal(3.14)) and isInt(f[i])!=True:
            st.append(c[i])
    return(st)

def categorical(df):
    c=df.columns
    catego=[]
    
    for i in range(0,len(c),1):
        if c[i].unique()<50:
              catego.append(c(i))
    return(catego)     

from pyspark.sql.functions import col
# esto funciona bien 

def castype(df): # esta funcion determina la categoria de cada una de las columnas 
    f=df.first() #primera fila del df
    c=df.columns 
    inte=catInt(df)
    flo=catFlo(df)
    st=catstring(df)
    for i in st:
        df.select(col("%s"%(i)).cast('string').alias("%s"%(i)))
    for j in inte:
        df.select(col("%s"%(j)).cast('int').alias("%s"%(j)))
    for k in flo:
        df.select(col("%s"%(k)).cast('float').alias("%s"%(k)))
    return(df)
    

#creacion variable objetivo

## se binariza el historial de los estados peso talla y se elimina la forma categorica de estos 
## Segun el comportamiento de las columnas estado peso talla se crea la columna objetivo que representa el historial de reincidencia

In [0]:
def ept(df):# devuelve el df con las columnas ept mapeados con los codigos
    p=df.select(filtro1(df, 'EstadoPesoTalla', 'IdBeneficiario')) # selecciona solo las columnas EstadoPesoTalla
    eptt=filtro2(df, 'EstadoPesoTalla') # lista de las columnas "estado peso talla" 
    p1=p.drop(*filtro2(p, 'IdBeneficiarioT')) #elimina IdBeneficiarioT
    listaEPT=p1.columns
    soloept=filtro2(p, 'EstadoPesoTalla')
    listaEPT=len(soloept)
    
    def somefunc(value):
        if value== 'Peso adecuado para la talla': 
                  return int(0)
        elif value=='Riesgo de desnutrición aguda':
            return int(0)
        elif value=='Desnutricion Aguda Moderada':
            return int(1)
        elif value=='Riesgo de Sobrepeso':
            return int(0)
        elif value=='Desnutricion aguda severa':
            return int(1)
        elif value=='Obesidad':
            return int(0)
        elif value=='Sobrepeso':
            return int(0)
        else:
            return int(0)
    udfsomefunc = F.udf(somefunc, IntegerType())
    EPTs=[]
    for c in (range(1,listaEPT+1)):
         EPTs.append('EPT%s'%(c))
    for i in range(listaEPT):
        df= df.withColumn(EPTs[i], udfsomefunc(df[soloept[i]]))
        #df= df.drop(*eptt)
    return   df

In [0]:
def reincidenciaff2(df):#
    soloept=filtro2(df,'EPT')# se crea una lista con las columnas estado peso talla binario
    c=df.select(soloept) #se crea el data frame con dichas columnas 
    listu = c.collect() #se obtienen todas las filas de dicho df 
    s=len(listu[1])# la cantidad de columnas 
    x=range(s,0,-1)
    l=len(listu) #la dimension de la matriz 
    t=range(0,l,1)
    ant=0
    rein=[]
    for j in t :
        r1 = False # formatea el estado de la bandera para cada fila
        for i in x: # se recorre las filas 

            if i==s:# primera iteracion de la ultima a la primera

                ant=listu[j][i-1] # primera iteracion ant = al valor ultima toma

            else:
                #print(0)
                if ant!=listu[j][i-1]:# si el anterior es diferente al actual y si r1 = true : r2=true y hay incidencia 
                    #print(ant)

                    if r1== True :
                        rein.append(1)
                        break  
                    ant=listu[j][i-1] # se actualiza el anterior
                    r1=True
            if s-i==s-1:# si el ultimo elemento es igual al penultimo elemento
                rein.append(0)
                break
            
    
    
    Ids=df.select('IdBeneficiario').rdd.flatMap(lambda x: x).collect() # se obtiene el Id de los usuarios como lista
    

    daa={'IdBeneficiarion': Ids, 'Reincidencia': rein } #se unen ambas listas el id y el estado de reincidencia
    dfp=pd.DataFrame(daa)# se crea el dataframe en pandas
    mySchema = StructType([StructField("IdBeneficiarion", StringType(), True)\
                           , StructField("Reincidencia",IntegerType(),True)])# se establece el schema

    dff = spark.createDataFrame(dfp, schema=mySchema)# se crea el dataframe en spark 


    merged_df=df.join(dff,df.IdBeneficiario ==  dff.IdBeneficiarion,"inner") # se hace el join entre el df que entra y el df que contiene el estado de reincidencia bajo la llave del ID 
    merged_df=merged_df.drop(merged_df.IdBeneficiarion) # se elimina la columna duplicada del ID 
    return(merged_df)



In [0]:
def balancing(df):# funcion que balancea 
    balancingRatio = df.filter(col("Reincidencia") == 1).count() / df.count()
    calculateWeights = udf(lambda x: 1 * balancingRatio if x == 0 else (1 * (1.0 - balancingRatio)), DoubleType())
    weightedDataset = df.withColumn("Reincidenciaw", calculateWeights("Reincidencia"))
    return(weightedDataset)

In [0]:
nulls(raw_datax)

In [0]:
poc=filtro4(raw_datax,17,0.8)
pathpoc="dbfs:/dbfs/mnt/poc17.parquet"
poc.write.mode("overwrite").parquet(pathpoc)

In [0]:
pathpoc="dbfs:/dbfs/mnt/poc17.parquet"
poc = spark.read.options(inferSchema='True',header='True').format("parquet").load(pathpoc)


Balanceo del dataframe

In [0]:
path="dbfs:/dbfs/mnt/p17rfinal.parquet"
prueba = spark.read.options(inferSchema='True',header='True').format("parquet").load(path)

In [0]:

def reincidentes(df):# esta funcion devuelve el dataframe de solo los sujetos reincidentes 
    #df=df.select([when(col(c)=="","Null").otherwise(col(c)).alias(c) for c in df.columns]) #llena las columnas con null
    df1=ept(df)
    df2=reincidenciaff2(df1)
    df3=df2.filter(df2.Reincidencia==1)
    df4=df2.filter(df2.Reincidencia==0)
    return(df3)


#### Etapa de String indexer OneHot encoding y vector assembler

Objetivo: volver las variables tipo string a una forma numérica y todas las categoricas hacerles OHE para crear el vector features

In [0]:
def features(df): # esta funcion le entra un df y crea un nuevo dataframe con el OHE de las columnas categoricas y devuelve un df con el el vector features que entra a los modelos de ML 
    
    x=catstring(df)
    lista=[]
    listinx=[]
    for i in x :
        lista.append("num_"+str(i))# se crea la lista con el nombre de las columnas que se vuelven de string a num
    y=lista
    stringindexer_stages =[StringIndexer(inputCols= x, outputCols= y).setHandleInvalid("skip")] # toma todas las columnas que son string 
    listunt=catInt(df) # lista con las columnas que son int 
    #listunt.remove("Reincidencia")
    listaohc=[]
    for i in y :# creacion de los nombrede las columnas a las que se les hace OHE
            listaohc.append("vec"+str(i))
    
    onehotencoder_stages = [OneHotEncoder( inputCols=y, outputCols=listaohc, handleInvalid='error')]#, dropLast=True)
    vectorassembler_stage = VectorAssembler(inputCols=listaohc, outputCol='features')
    all_stages = stringindexer_stages + onehotencoder_stages + [vectorassembler_stage]
    pipeline = Pipeline(stages=all_stages).fit(df).transform(df)
    final_columns = ["Idbeneficiario"]+['features'] + ['Reincidenciaw']+['Reincidencia'] #listunt +
    df_features = pipeline.select(final_columns)
    return (df_features)

# construccion de las 3 sabanas de datos:
- La primera con todas las variables 
- La segunda con el 75% de las variables 
- La tercera con el 50% de las variables

In [0]:
p1705=filtro4(raw_datax,17,0)
p1005=filtro4(raw_datax,10,0.2)
p605=filtro4(raw_datax,6,0.3)

#creacion de las columnas ept 
p17ept=ept(p1705)
p1005ept=ept(p1005)
p605ept=ept(p605)

# creacion de la columna reincidencia
p1705r=reincidenciaff2(p17ept)
p1005eptr=reincidenciaff2(p1005ept)
p605eptr=reincidenciaff2(p605ept)

#arreglar schema categorizando cada una
p1705rcast=castype(p1705r)
p1005rcast=castype(p1005eptr)
p605cast=castype(p605eptr)

p1705rcast=balancing(p1705rcast)
p1005rcast=balancing(p1005rcast)
p605cast=balancing(p605cast)
#Creacion del dataframe final con la columna features que contiene la informacion de todas las columnas 


path_to_datap1705r="dbfs:/dbfs/mnt/p1705r.parquet"
path_to_datap605eptr="dbfs:/dbfs/mnt/p605eptr.parquet"
path_to_datap1005eptr="dbfs:/dbfs/mnt/p1005eptr.parquet"

p1705rcast.write.mode("overwrite").parquet(path_to_datap1705r)
p1005rcast.write.mode("overwrite").parquet(path_to_datap605eptr)
p605cast.write.mode("overwrite").parquet(path_to_datap1005eptr)

In [0]:
path_to_datap1705r="dbfs:/dbfs/mnt/p1705r.parquet"
path_to_datap605eptr="dbfs:/dbfs/mnt/p605eptr.parquet"
path_to_datap1005eptr="dbfs:/dbfs/mnt/p1005eptr.parquet"

p1705rcast = spark.read.options(inferSchema='True',header='True').format("parquet").load(path_to_datap1705r)
p605cast = spark.read.options(inferSchema='True',header='True').format("parquet").load(path_to_datap605eptr)
p1005rcast = spark.read.options(inferSchema='True',header='True').format("parquet").load(path_to_datap1005eptr)

In [0]:
p17feat=features(p1705rcast)
p6feat=features(p605cast)
p10feat=features(p1005rcast)

path_17feat="dbfs:/dbfs/mnt/p17feat.parquet"
path_6feat="dbfs:/dbfs/mnt/p6feat.parquet"
path_10feat="dbfs:/dbfs/mnt/p10feat.parquet"

p17feat.write.mode("overwrite").parquet(path_17feat)
p10feat.write.mode("overwrite").parquet(path_10feat)
p6feat.write.mode("overwrite").parquet(path_6feat)

In [0]:
path_17feat="dbfs:/dbfs/mnt/p17feat.parquet"
path_6feat="dbfs:/dbfs/mnt/p6feat.parquet"
path_10feat="dbfs:/dbfs/mnt/p10feat.parquet"

p17feat = spark.read.options(inferSchema='True',header='True').format("parquet").load(path_17feat)
path_6feat = spark.read.options(inferSchema='True',header='True').format("parquet").load(path_10feat)
p10feat = spark.read.options(inferSchema='True',header='True').format("parquet").load(path_6feat)

In [0]:
print(p1705rcast.count())
print(p605cast.count())
print(p1005rcast.count())

In [0]:
nulls(p1705rcast)

In [0]:
nulls(p605cast)

In [0]:
nulls(p1005rcast)

In [0]:
pocept=ept(poc)
reinpoc=reincidenciaff2(pocept)
pocreiffcas=castype(reinpoc)
pocbal=balancing(pocreiffcas)
pocfeat=features(pocbal)
display(pocfeat)

%md tres modelos de machine learning 

- regresion Lineal
- Regresion Logistica
- random Forest

In [0]:
%sh

pip install mlflow

/databricks/python_shell/scripts/PythonShell.py configure

Requirement already satisfied: mlflow in /databricks/python3/lib/python3.8/site-packages (1.27.0)
Requirement already satisfied: pytz in /databricks/python3/lib/python3.8/site-packages (from mlflow) (2020.5)
Requirement already satisfied: Flask in /databricks/python3/lib/python3.8/site-packages (from mlflow) (2.1.2)
Requirement already satisfied: scipy in /databricks/python3/lib/python3.8/site-packages (from mlflow) (1.6.2)
Requirement already satisfied: alembic in /databricks/python3/lib/python3.8/site-packages (from mlflow) (1.8.0)
Requirement already satisfied: pandas in /databricks/python3/lib/python3.8/site-packages (from mlflow) (1.2.4)
Requirement already satisfied: sqlparse>=0.3.1 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (0.4.2)
Requirement already satisfied: protobuf>=3.12.0 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (3.17.2)
Requirement already satisfied: gitpython>=2.1.0 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (3.1.27)
Requirement already satisfied: numpy in /databricks/python3/lib/python3.8/site-packages (from mlflow) (1.19.2)
Requirement already satisfied: databricks-cli>=0.8.7 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (0.17.0)
Requirement already satisfied: packaging in /databricks/python3/lib/python3.8/site-packages (from mlflow) (20.9)
Requirement already satisfied: requests>=2.17.3 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (2.25.1)
Requirement already satisfied: cloudpickle in /databricks/python3/lib/python3.8/site-packages (from mlflow) (2.1.0)
Requirement already satisfied: click>=7.0 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (8.1.3)
Requirement already satisfied: sqlalchemy>=1.4.0 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (1.4.39)
Requirement already satisfied: gunicorn in /databricks/python3/lib/python3.8/site-packages (from mlflow) (20.1.0)
Requirement already satisfied: docker>=4.0.0 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (5.0.3)
Requirement already satisfied: querystring-parser in /databricks/python3/lib/python3.8/site-packages (from mlflow) (1.2.4)
Requirement already satisfied: pyyaml>=5.1 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (6.0)
Requirement already satisfied: importlib-metadata!=4.7.0,>=3.7.0 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (4.12.0)
Requirement already satisfied: entrypoints in /databricks/python3/lib/python3.8/site-packages (from mlflow) (0.3)
Requirement already satisfied: prometheus-flask-exporter in /databricks/python3/lib/python3.8/site-packages (from mlflow) (0.20.2)
Requirement already satisfied: oauthlib>=3.1.0 in /databricks/python3/lib/python3.8/site-packages (from databricks-cli>=0.8.7->mlflow) (3.2.0)
Requirement already satisfied: tabulate>=0.7.7 in /databricks/python3/lib/python3.8/site-packages (from databricks-cli>=0.8.7->mlflow) (0.8.10)
Requirement already satisfied: pyjwt>=1.7.0 in /databricks/python3/lib/python3.8/site-packages (from databricks-cli>=0.8.7->mlflow) (2.4.0)
Requirement already satisfied: six>=1.10.0 in /databricks/python3/lib/python3.8/site-packages (from databricks-cli>=0.8.7->mlflow) (1.15.0)
Requirement already satisfied: websocket-client>=0.32.0 in /databricks/python3/lib/python3.8/site-packages (from docker>=4.0.0->mlflow) (1.3.3)
Requirement already satisfied: gitdb<5,>=4.0.1 in /databricks/python3/lib/python3.8/site-packages (from gitpython>=2.1.0->mlflow) (4.0.9)
Requirement already satisfied: smmap<6,>=3.0.1 in /databricks/python3/lib/python3.8/site-packages (from gitdb<5,>=4.0.1->gitpython>=2.1.0->mlflow) (5.0.0)
Requirement already satisfied: zipp>=0.5 in /databricks/python3/lib/python3.8/site-packages (from importlib-metadata!=4.7.0,>=3.7.0->mlflow) (3.8.1)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.17.3->mlflow) (2020.12.5)
Require

In [0]:
import mlflow
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import RegressionEvaluator, BinaryClassificationEvaluator

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

/bin/bash: 2: command not found
Collecting mlflow
 Downloading mlflow-1.27.0-py3-none-any.whl (17.9 MB)
Requirement already satisfied: pytz in /databricks/python3/lib/python3.8/site-packages (from mlflow) (2020.5)
Collecting Flask
 Downloading Flask-2.1.2-py3-none-any.whl (95 kB)
Requirement already satisfied: scipy in /databricks/python3/lib/python3.8/site-packages (from mlflow) (1.6.2)
Collecting alembic
 Downloading alembic-1.8.0-py3-none-any.whl (209 kB)
Requirement already satisfied: pandas in /databricks/python3/lib/python3.8/site-packages (from mlflow) (1.2.4)
Collecting sqlparse>=0.3.1
 Downloading sqlparse-0.4.2-py3-none-any.whl (42 kB)
Requirement already satisfied: protobuf>=3.12.0 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (3.17.2)
Collecting gitpython>=2.1.0
 Downloading GitPython-3.1.27-py3-none-any.whl (181 kB)
Requirement already satisfied: numpy in /databricks/python3/lib/python3.8/site-packages (from mlflow) (1.19.2)
Collecting databricks-cli>=0.8.7
 Downloading databricks-cli-0.17.0.tar.gz (81 kB)
Requirement already satisfied: packaging in /databricks/python3/lib/python3.8/site-packages (from mlflow) (20.9)
Requirement already satisfied: requests>=2.17.3 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (2.25.1)
Collecting cloudpickle
 Downloading cloudpickle-2.1.0-py3-none-any.whl (25 kB)
Collecting click>=7.0
 Downloading click-8.1.3-py3-none-any.whl (96 kB)
Collecting sqlalchemy>=1.4.0
 Downloading SQLAlchemy-1.4.39-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.6 MB)
Collecting gunicorn
 Downloading gunicorn-20.1.0-py3-none-any.whl (79 kB)
Collecting docker>=4.0.0
 Downloading docker-5.0.3-py2.py3-none-any.whl (146 kB)
Collecting querystring-parser
 Downloading querystring_parser-1.2.4-py2.py3-none-any.whl (7.9 kB)
Collecting pyyaml>=5.1
 Downloading PyYAML-6.0-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (701 kB)
Collecting importlib-metadata!=4.7.0,>=3.7.0
 Downloading importlib_metadata-4.12.0-py3-none-any.whl (21 kB)
Requirement already satisfied: entrypoints in /databricks/python3/lib/python3.8/site-packages (from mlflow) (0.3)
Collecting prometheus-flask-exporter
 Downloading prometheus_flask_exporter-0.20.2-py3-none-any.whl (18 kB)

metricName: pyspark.ml.param.Param[MulticlassClassificationEvaluatorMetricType] = Param(parent='undefined', name='metricName', doc='metric name in evaluation (f1|accuracy|weightedPrecision|weightedRecall|weightedTruePositiveRate| weightedFalsePositiveRate|weightedFMeasure|truePositiveRateByLabel| falsePositiveRateByLabel|precisionByLabel|recallByLabel|fMeasureByLabel| logLoss|hammingLoss)')

#### Regresión lineal

In [0]:
def reglin(feat):    
    model_dff1=feat
    train_df,test_df=model_dff1.randomSplit([0.5,0.5])
    lin_Reg=LinearRegression(featuresCol='features',labelCol='Reincidencia',weightCol="Reincidenciaw")
    lrparamGrid = (ParamGridBuilder()
             .addGrid(lin_Reg.regParam, [ 0.1, 0.5, 1.0, 2.0])

             .addGrid(lin_Reg.elasticNetParam, [0.0, 0.5, 0.75, 1.0])

             .addGrid(lin_Reg.maxIter, [1, 5, 10, 20, 30])

             .build())
    #evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="Reincidencia", weightCol="Reincidenciaw", metricName="rmse")
    evaluator = BinaryClassificationEvaluator( labelCol='Reincidencia',weightCol="Reincidenciaw")
    cv_LiR = CrossValidator(estimator = lin_Reg, 
                      estimatorParamMaps = lrparamGrid,
                      evaluator = evaluator,
                      numFolds = 5)

    crossvalidation_mod = cv_LiR.fit(train_df)# fiting cross validation 
    #lrcvSummary = crossvalidation_mod.bestModel.summary # resumen de los parametros de mejor modelo
    #prediccion en el conjunto de entrenamiento
    pred_train = crossvalidation_mod.transform(train_df)

    #data frame con la prediccion en el conjunto de test
    lrpredictions =crossvalidation_mod.transform(test_df) # evaluacion del modelo 

    
    #label_pred_train = pred_train.select('Idbeneficiario','Reincidencia', 'prediction')
    
    label_pred_test = lrpredictions.select('Idbeneficiario','Reincidencia','Reincidenciaw', 'prediction')


    
    
    #Confusion matrix from test data
    tp = label_pred_test[(lrpredictions.Reincidencia == 1) & (lrpredictions.prediction > 0.5)].count()
    tn = label_pred_test[(lrpredictions.Reincidencia == 0) & (lrpredictions.prediction< 0.5)].count()
    fp = label_pred_test[(lrpredictions.Reincidencia == 0) & (lrpredictions.prediction > 0.5)].count()
    fn = label_pred_test[(lrpredictions.Reincidencia == 1) & (lrpredictions.prediction < 0.5)].count()
    
    #indicadores
    coef=evaluator.evaluate(lrpredictions)
    r2=evaluator.setMetricName('r2').evaluate(lrpredictions)# R^2
    coef=print('Intercept: ', crossvalidation_mod.bestModel.intercept)
    accuracy=float((tp+tn) /(label_pred_test.count()))
    recall = float(tp)/(tp + fn)
    precision = float(tp) / (tp +fp)
    f1=2*(precision*recall/(precision+recall))
    rmse = RegressionEvaluator(labelCol="Reincidenciaw", predictionCol="prediction", metricName="rmse")
    rmse = rmse.evaluate(label_pred_test)
    score = {"tp":tp,"tn":tn,"fp":fp,"fn":fn,"accuracy": accuracy, "recall": recall, "precision": precision,"f1":f1}
    mae = RegressionEvaluator(labelCol="Reincidencia", predictionCol="prediction", metricName="mae")
    mae = mae.evaluate(label_pred_test)
    r2 = RegressionEvaluator(labelCol="Reincidencia", predictionCol="prediction", metricName="r2")
    r2 = r2.evaluate(label_pred_test)


    print("MAE: ", mae)
    print("R-squared: ", r2)
    print(score)
    print('RMSE:',rmse)
    
    return(label_pred_test)



In [0]:
reglog(pocfeat)

In [0]:
path_17feat="dbfs:/dbfs/mnt/p17feat.parquet"
p17feat = spark.read.options(inferSchema='True',header='True').format("parquet").load(path_17feat)
p17reglin=reglin(p17feat)
path_17reglin="dbfs:/dbfs/mnt/p17reglin.parquet"
p17reglin.write.mode("overwrite").parquet(path_17reglin)

path_6feat="dbfs:/dbfs/mnt/p6feat.parquet"
p6feat = spark.read.options(inferSchema='True',header='True').format("parquet").load(path_10feat)
p6reglin=reglin(p6feat)
path_6reglin="dbfs:/dbfs/mnt/p6reglin.parquet"
p6reglin.write.mode("overwrite").parquet(path_6reglin)

path_10feat="dbfs:/dbfs/mnt/p10feat.parquet"
p10feat = spark.read.options(inferSchema='True',header='True').format("parquet").load(path_6feat)
p10reglin=reglin(p10feat)
path_10reglin="dbfs:/dbfs/mnt/p10reglin.parquet"
p10reglin.write.mode("overwrite").parquet(path_10reglin)



/databricks/spark/python/pyspark/ml/util.py:894: UserWarning: MLlib could not reach the MLflow server at tracking URI: databricks
Exception: You haven't configured the CLI yet! Please configure by entering `/databricks/python_shell/scripts/PythonShell.py configure`
 warnings.warn(
/databricks/spark/python/pyspark/ml/util.py:92: UserWarning: CrossValidator_8c6c1f484430 fit call failed but some spark jobs may still running for unfinished trials. To address this issue, you should enable pyspark pinned thread mode.
 warnings.warn("{} fit call failed but some spark jobs "

--------------------------------------------------------------------------- 
 IllegalArgumentException Traceback (most recent call last)
 <command-3428115858088589> in <module> 
 1 path_17feat = "dbfs:/dbfs/mnt/p17feat.parquet" 
 2 p17feat = spark . read . options ( inferSchema = 'True' , header = 'True' ) . format ( "parquet" ) . load ( path_17feat ) 
 ----> 3 p17reglin = reglin ( p17feat ) 
 4 path_17reglin = "dbfs:/dbfs/mnt/p17reglin.parquet" 
 5 p17reglin . write . mode ( "overwrite" ) . parquet ( path_17reglin ) 

 <command-3410559751730731> in reglin (feat) 
 18 numFolds = 5)
 19 
 ---> 20 crossvalidation_mod = cv_LiR . fit ( train_df ) # fiting cross validation 
 21 #lrcvSummary = crossvalidation_mod.bestModel.summary # resumen de los parametros de mejor modelo 
 22 #prediccion en el conjunto de entrenamiento 

 /databricks/python_shell/dbruntime/MLWorkloadsInstrumentation/_pyspark.py in patched_method (self, *args, **kwargs) 
 28 call_succeeded = False 
 29 try : 
 ---> 30 result = original_method ( self , * args , ** kwargs ) 
 31 call_succeeded = True 
 32 return result

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 159 return self . copy ( params ) . _fit ( dataset ) 
 160 else : 
 --> 161 return self . _fit ( dataset ) 
 162 else : 
 163 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/tuning.py in _fit (self, dataset) 
 711 subModels [ i ] [ j ] = subModel
 712 
 --> 713 _cancel_on_failure ( dataset . _sc , self . uid , sub_task_failed , calculate_metrics ) 
 714 # END-EDGE 
 715 

 /databricks/spark/python/pyspark/ml/util.py in _cancel_on_failure (sc, uid, sub_task_failed, f) 
 94 "issue, you should enable pyspark pinned thread mode." 
 95 .format(uid))
 ---> 96 raise e
 97 
 98 old_job_group = sc . getLocalProperty ( "spark.jobGroup.id" ) 

 /databricks/spark/python/pyspark/ml/util.py in _cancel_on_failure (sc, uid, sub_task_failed, f) 
 88 if os . environ . get ( "PYSPARK_PIN_THREAD" , "false" ) . lower ( ) != "true" : 
 89 try : 
 ---> 90 return f ( ) 
 91 except Exception as e : 
 92 warnings.warn("{} fit call failed but some spark jobs "

 /databricks/spark/python/pyspark/ml/util.py in wrapper (*args, **kwargs) 
 148 
 149 ipython . events . register ( "post_run_cell" , on_cancel ) 
 --> 150 return f ( * args , ** kwargs ) 
 151 
 152 return wrapper

 /databricks/spark/python/pyspark/ml/tuning.py in calculate_metrics () 
 705 return task ( ) 
 706 
 --> 707 for j , metric , subModel in pool . imap_unordered ( run_task , tasks ) : 
 708 metrics [ j ] += ( metric / nFolds ) 
 709 metrics_all [ i ] [ j ] = metric

 /usr/lib/python3.8/multiprocessing/pool.py in next (self, timeout) 
 866 if success : 
 867 return value
 --> 868 raise value
 869 
 870 __next__ = next # XXX 

 /usr/lib/python3.8/multiprocessing/pool.py in worker (inqueue, outqueue, initializer, initargs, maxtasks, wrap_exception) 
 123 job , i , func , args , kwds = task
 124 try : 
 --> 125 result = ( True , func ( * args , ** kwds ) ) 
 126 except Exception as e : 
 127 if wrap_exception and func is not _helper_reraises_exception : 

 /databricks/spark/python/pyspark/ml/tuning.py in run_task (task) 
 703 if sub_task_failed [ 0 ] : 
 704 raise RuntimeError ( "Terminate this task because one of other task failed." ) 
 --> 705 return task ( ) 
 706 
 707 for j , metric , subModel in pool . imap_unordered ( run_task , tasks ) : 

 /databricks/spark/python/pyspark/ml/tuning.py in singleTask () 
 73 # `MetaAlgorithmReadWrite.getAllNestedStages`, make it return 
 74 # all nested stages and evaluators 
 ---> 75 metric = eva . evaluate ( model . transform ( validation , epm [ index ] ) ) 
 76 return index , metric , model if collectSubModel else None 
 77 

 /databricks/spark/python/pyspark/ml/evaluation.py in evaluate (self, dataset, params) 
 82 return self . copy ( params ) . _evaluate ( dataset ) 
 83 else : 
 ---> 84 return self . _evaluate ( da

#### Regresion Logística

In [0]:
from pyspark.ml.classification import LogisticRegression
def reglogist(feat):
    #se toman solo las columnas features y reincidencia balanceado
    model_df=feat
    
    #divison aleatoria
    training_df,test_df=model_df.randomSplit([0.5,0.5]) 
    
    #aplicacion de la regresion logistica 
    log_reg = LogisticRegression(featuresCol='features',labelCol='Reincidencia',weightCol="Reincidenciaw")
    
    #modelo al set de entranmiento
    #log_reg.fit(training_df)#reincidencia
    
    # evaluador de desempeño binario 
    #evaluator_LR = BinaryClassificationEvaluator( labelCol='probability')
    evaluator_LR = BinaryClassificationEvaluator( labelCol='Reincidencia',weightCol="Reincidenciaw")
    
    # Creacion ParamGrid para Cross Validation
    paramGrid_LR = (ParamGridBuilder()
                 .addGrid(log_reg.regParam, [ 0.1, 0.5, 1.0, 2.0])
                 .addGrid(log_reg.elasticNetParam, [0.0, 0.5, 0.75, 1.0])
                 .addGrid(log_reg.maxIter, [1, 5, 10, 20, 40])
                 .build())


    pipelinelr = Pipeline(stages=[log_reg])

    # Crea 5-fold CrossValidator
    cv_LR = CrossValidator(estimator = pipelinelr,
                          estimatorParamMaps = paramGrid_LR,
                          evaluator = evaluator_LR,
                          numFolds = 5)
    
    
    # Fit cross-validation model Run cross validations.
    cvModel_lr = cv_LR.fit(model_df)
    
    #Prediction
    
    #Prediction on training data
    pred_train = cvModel_lr.transform(training_df)
    
    #Prediction on test data
    predictions_lr = cvModel_lr.transform(test_df)
    
    lrcvSummary = cvModel_lr.bestModel.summary
    
    #evaluacion del entrenamiento de la prediccion 
    #train_results=log_reg.evaluate(training_df).predictions#reincidencia

    # evaluacion del modelo
    #results=log_reg.evaluate(test_df).predictions#reincidencia
    
    
    
    
    
    #Confusion matrix from test data

    
    label_pred_test = predictions_lr.select('Idbeneficiario','Reincidencia','Reincidenciaw', 'prediction')
    
    
    # Evaluacion del  modelo indicadores
    
    tp = label_pred_test[(predictions_lr.Reincidencia == 1) & (predictions_lr.prediction > 0.5)].count()
    tn = label_pred_test[(predictions_lr.Reincidencia == 0) & (predictions_lr.prediction< 0.5)].count()
    fp = label_pred_test[(predictions_lr.Reincidencia == 0) & (predictions_lr.prediction > 0.5)].count()
    fn = label_pred_test[(predictions_lr.Reincidencia == 1) & (predictions_lr.prediction < 0.5)].count()
    
    accuracy=float((tp+tn) /(label_pred_test.count()))
    recall = float(tp)/(tp + fn)
    precision = float(tp) / (tp +fp)
    f1=2*(precision*recall/(precision+recall))
    
    ROCTest=evaluator.setMetricName('areaUnderROC').evaluate(label_pred_test)
    rms1=evaluator.setMetricName('rmse').evaluate(label_pred_test)
    mae1=evaluator.setMetricName('mae').evaluate(label_pred_test)
    r21=evaluator.setMetricName('r2').evaluate(label_pred_test)
    print("AUROC:",ROCTest)
    print("MAE: ", mae1)
    print("R-squared: ", r21)
    print('RMSE:',rms1)
    
    
    rmse = RegressionEvaluator(labelCol="Reincidenciaw", predictionCol="prediction", metricName="rmse")
    rmse = rmse.evaluate(label_pred_test)
    score = {"tp":tp,"tn":tn,"fp":fp,"fn":fn,"accuracy": accuracy, "recall": recall, "precision": precision,"f1":f1}
    mae = RegressionEvaluator(labelCol="Reincidencia", predictionCol="prediction", metricName="mae")
    mae = mae.evaluate(label_pred_test)
    r2 = RegressionEvaluator(labelCol="Reincidencia", predictionCol="prediction", metricName="r2")
    r2 = r2.evaluate(label_pred_test)
    areaUnderROCtest = RegressionEvaluator(labelCol="Reincidenciaw", predictionCol="prediction", metricName="areaUnderROC")
    areaUnderROCtest = areaUnderROCtest.evaluate(label_pred_test)
    
    print("AUROC:",areaUnderROCtest)
    print("MAE: ", mae)
    print("R-squared: ", r2)
    print(score)
    print('RMSE:',rmse)
    
    return(label_pred_test)





In [0]:
reglogist(pocfeat)

In [0]:
path_17feat="dbfs:/dbfs/mnt/p17feat.parquet"
p17feat = spark.read.options(inferSchema='True',header='True').format("parquet").load(path_17feat)
p17logreg=reglogist(p17feat)
path_17logreg="dbfs:/dbfs/mnt/p17logreg.parquet"
p17logreg.write.mode("overwrite").parquet(path_17logreg)

path_6feat="dbfs:/dbfs/mnt/p6feat.parquet"
p6feat = spark.read.options(inferSchema='True',header='True').format("parquet").load(path_10feat)
p6logreg=reglogist(p6feat)
path_6logreg="dbfs:/dbfs/mnt/p6logreg.parquet"
p6logreg.write.mode("overwrite").parquet(path_6logreg)

path_10feat="dbfs:/dbfs/mnt/p10feat.parquet"
p10feat = spark.read.options(inferSchema='True',header='True').format("parquet").load(path_6feat)
p10logreg=reglogist(p10feat)
path_10logreg="dbfs:/dbfs/mnt/p10logreg.parquet"
p10logreg.write.mode("overwrite").parquet(path_10logreg)



#### Random Forest

In [0]:
def randomforest(feat):
    model_RF=feat
    
    #division aleatoria 
    train_df,tes_df=model_df.randomSplit([0.5,0.5])
    
    #estimador
    random_forest = RandomForestClassifier(featuresCol='features',labelCol='Reincidencia',weightCol="Reincidenciaw")
    
    
    # Evaluate model
    rfevaluator = BinaryClassificationEvaluator( labelCol='Reincidencia',weightCol="Reincidenciaw")
    #evaluator = BinaryClassificationEvaluator()
    
    # Creacion grid de parametrospara Cross Validation

    pipeline = Pipeline(stages=[rf_classifier])

    rfparamGrid = (ParamGridBuilder()
                   .addGrid(random_forest.maxDepth, [2, 5, 10, 20, 30])
                   .addGrid(random_forest.maxBins, [5, 10, 20, 40, 100])
                   .addGrid(random_forest.numTrees, [5, 10, 20, 40, 50])
                   .addGrid(random_forest.minInfoGain, [0.0, 0.1, 0.2, 0.3])
                 .build())



    #  5-fold CrossValidator
    rfcv = CrossValidator(estimator = pipeline,
                          estimatorParamMaps = rfparamGrid,
                          evaluator = rfevaluator,
                          numFolds = 5)
    
    #Fit cross-validation model
    crossvalidation_mod = rfcv.fit(model_RF)
    
    #prediction
    
    #en training
    pred_train = crossvalidation_mod.transform(train_df)
    
    #en test
    pred_test = crossvalidation_mod.transform(tes_df)
    
    label_pred_test = pred_test.select('Idbeneficiario','Reincidencia','Reincidenciaw', 'prediction')
    
    #Prediction performance
    
    # Evaluacion del  modelo indicadores
    
    tp = label_pred_test[(predictions_lr.Reincidencia == 1) & (predictions_lr.prediction > 0.5)].count()
    tn = label_pred_test[(predictions_lr.Reincidencia == 0) & (predictions_lr.prediction< 0.5)].count()
    fp = label_pred_test[(predictions_lr.Reincidencia == 0) & (predictions_lr.prediction > 0.5)].count()
    fn = label_pred_test[(predictions_lr.Reincidencia == 1) & (predictions_lr.prediction < 0.5)].count()
    
    
    #ROC
    
    ROCTest=evaluator.setMetricName('areaUnderROC').evaluate(label_pred_test)
    rms1=evaluator.setMetricName('rmse').evaluate(label_pred_test)
    mae1=evaluator.setMetricName('mae').evaluate(label_pred_test)
    r21=evaluator.setMetricName('r2').evaluate(label_pred_test)
    print("AUROC:",ROCTest)
    print("MAE: ", mae1)
    print("R-squared: ", r21)
    print('RMSE:',rms1)
    
    accuracy=float((tp+tn) /(label_pred_test.count()))
    recall = float(tp)/(tp + fn)
    precision = float(tp) / (tp +fp)
    f1=2*(precision*recall/(precision+recall))
    rmse = RegressionEvaluator(labelCol="Reincidenciaw", predictionCol="prediction", metricName="rmse")
    rmse = rmse.evaluate(label_pred_test)
    score = {"tp":tp,"tn":tn,"fp":fp,"fn":fn,"accuracy": accuracy, "recall": recall, "precision": precision,"f1":f1}
    mae = RegressionEvaluator(labelCol="Reincidencia", predictionCol="prediction", metricName="mae")
    mae = mae.evaluate(label_pred_test)
    r2 = RegressionEvaluator(labelCol="Reincidencia", predictionCol="prediction", metricName="r2")
    r2 = r2.evaluate(label_pred_test)
    areaUnderROCtest = RegressionEvaluator(labelCol="Reincidenciaw", predictionCol="prediction", metricName="areaUnderROC")
    areaUnderROCtest = areaUnderROCtest.evaluate(label_pred_test)
    
    print("AUROC:",areaUnderROCtest)
    print("MAE: ", mae)
    print("R-squared: ", r2)
    print('RMSE:',rmse)
    print(score)
    
    return(label_pred_test)
    
    
    
    
   
    

In [0]:
path_17feat="dbfs:/dbfs/mnt/p17feat.parquet"
p17feat = spark.read.options(inferSchema='True',header='True').format("parquet").load(path_17feat)
p17rf=randomforest(p17feat)
path_17rf="dbfs:/dbfs/mnt/p17rf.parquet"
p17rf.write.mode("overwrite").parquet(path_17rf)

path_6feat="dbfs:/dbfs/mnt/p6feat.parquet"
p6feat = spark.read.options(inferSchema='True',header='True').format("parquet").load(path_10feat)
p6rf=randomforest(p6feat)
path_6rf="dbfs:/dbfs/mnt/p6rf.parquet"
p6rf.write.mode("overwrite").parquet(path_6rf)

path_10feat="dbfs:/dbfs/mnt/p10feat.parquet"
p10feat = spark.read.options(inferSchema='True',header='True').format("parquet").load(path_6feat)
p10rf=randomforest(p10feat)
path_10rf="dbfs:/dbfs/mnt/p10rf.parquet"
p10rf.write.mode("overwrite").parquet(path_10rf)



In [0]:
randomforest(pocfeat)

In [0]:
random_forest.write().overwrite().save('dbfs:/mnt/gold/icbf-gai/Random_Forest_Classifier/') # asi se guarda el modelo final

In [0]:
spark2=SparkSession.builder.appName('random_forest').getOrCreate()
model_df4=p1705_df.select(['IdBeneficiario','features','Reincidencia'])

#division aleatoria 
train_df,tes_df=model_df.randomSplit([0.75,0.25])#EPT3

#aplicacion del arbol aleatorio 
rf_classifier=RandomForestClassifier(labelCol='Reincidencia',numTrees=50).fit(train_df)

#transformacion del clasificadoer 
rf_predictions=rf_classifier.transform(tes_df)
#evaluacion
resultsrf=rf_classifier.evaluate(tes_df).predictions
# evaluacion del acuracy
rf_accuracy=MulticlassClassificationEvaluator(labelCol='Reincidencia',metricName='accuracy').evaluate(rf_predictions)

#evaluacion de la precision ponderada
rf_precision=MulticlassClassificationEvaluator(labelCol='Reincidencia',metricName='weightedPrecision').evaluate(rf_predictions)
#AUC
rf_auc=BinaryClassificationEvaluator(rawPredictionCol=resultsrf, labelCol= 'Reincidencia', metricName = 'areaUnderROC',).evaluate(rf_predictions)
 

#Metrics
print('El Accuracy Score con los datos de test es {0:.0%}'.format(rf_accuracy))
print('El Precision Rate Score con los datos de test es {0:.0%}'.format(rf_precision))
print('El AUC es {0:.0%}'.format(rf_auc) )

AUC ranges in value from 0 to 1. A model whose predictions are 100% wrong has an AUC of 0.0; one whose predictions are 100% correct has an AUC of 1.0.

AUC is desirable for the following two reasons:

AUC is scale-invariant. It measures how well predictions are ranked, rather than their absolute values.
AUC is classification-threshold-invariant. It measures the quality of the model's predictions irrespective of what classification threshold is chosen.

In [0]:
# matriz de confusion 
tp = resultsrf[(resultsrf.Reincidencia == 1) & (resultsrf.prediction== 1)].count()
tn = resultsrf[(resultsrf.Reincidencia == 0) & (resultsrf.prediction== 0)].count()
fp = resultsrf[(resultsrf.Reincidencia == 0) & (resultsrf.prediction== 1)].count()
fn = resultsrf[(resultsrf.Reincidencia == 1) & (resultsrf.prediction== 0)].count()

print("*** Resultados Regresion Logística *** \n")

print(f'True Positive: {tp}')
print(f'True Negative: {tn}')
print(f'False Positive: {fp}')
print(f'False Negative: {fn}')

accuracy=float((tp+tn) /(resultsrf.count()))
print(f'Accuracy: {accuracy}')

#recall = float(tp)/(tp + fn)
#print(f'Recall: {recall}')

#precision = float(tp) / (tp +fp)
#print(f'Precision: {precision}')

In [0]:
join_psdf =model_df4.join(rf_predictions,on='features')
dfFinal = join_psdf.select('IdBeneficiario','prediction')

### Cross validation

In [0]:


# Evaluate model
rfevaluator = BinaryClassificationEvaluator( labelCol='reincidencia')

# Creacion ParamGrid para Cross Validation

pipeline = Pipeline(stages=[rf_classifier])

rfparamGrid = (ParamGridBuilder()
             #.addGrid(rf.maxDepth, [2, 5, 10, 20, 30])
               .addGrid(rf_classifier.maxDepth, [2, 5, 10, 20, 30])
             #.addGrid(rf.maxBins, [10, 20, 40, 80, 100])
               .addGrid(rf_classifier.maxBins, [5, 10, 20, 40, 80, 100])
             #.addGrid(rf.numTrees, [5, 20, 50, 100, 500])
               .addGrid(rf_classifier.numTrees, [5, 20, 50, 100, 500])
             .build())



#  5-fold CrossValidator
rfcv = CrossValidator(estimator = pipeline,
                      estimatorParamMaps = rfparamGrid,
                      evaluator = rfevaluator,
                      numFolds = 5)



# se ejecuta cross validations.


rfcvModel = rfcv.fit(train_df)
print(rfcvModel)
rfcvModel
rfpredictions = rfcvModel.transform(tes_df)

# cvModel uses the best model found from the Cross Validation
# Evaluacion del modelo
print('RMSE:', rfevaluator.evaluate(rfpredictions))

> Se guarda el modelo RandomForest

In [0]:
rf_classifier.write().overwrite().save('dbfs:/mnt/gold/icbf-gai/Random_Forest_Classifier/')

> Se hace predicción para toda la base:

In [0]:
#Transformacion del Clasificador:
rf_predictionsTotal=rf_classifier.transform(model_df)

#Evaluación del Acuracy:
rf_accuracyTotal = MulticlassClassificationEvaluator(labelCol='reincidencia',metricName='accuracy').evaluate(rf_predictionsTotal)

#Evaluación de la Precision Ponderada
rf_precisionTotal = MulticlassClassificationEvaluator(labelCol='reincidencia',metricName='weightedPrecision').evaluate(rf_predictionsTotal)

#Metrics
print('El Accuracy Score con los datos de toda la base es {0:.0%}'.format(rf_accuracyTotal))
print('El Precision Rate Score con los datos de toda la base es {0:.0%}'.format(rf_precisionTotal))

In [0]:
rf_predictionsTotal.groupBy('prediction').count().show()

In [0]:
join_psdf =model_df4.join(rf_predictionsTotal,on='features')
dfPredicciones = join_psdf.select('IdBeneficiario','prediction')

In [0]:
#---------------------------- Borrar Histórico ----------------------------------------#
dbfsWriteDelta1 = "dbfs:/mnt/gold/icbf-gai/BaseModeloProbabilisticoPredicciones/"
try:
    dbutils.fs.ls(dbfsWriteDelta1)
    print(" *** Borrando Directorio *** ")
    files = dbutils.fs.ls(dbfsWriteDelta1)
    for f in files:
        f.path
        dbutils.fs.rm(f.path, recurse=True)
except:
    print(" *** No Existe Directorio *** ")
    pass
#---------------------------- Directorio de Salida ------------------------------------#
pathSave = dbfsWriteDelta1
nombreTabla = "dbo.dfPredicciones"
dfPredicciones.write \
.format("delta") \
.mode("overwrite") \
.option("overwriteSchema", "true") \
.save(f"{pathSave}") 
print(" *** Guardado en el DataLake *** ")
spark.sql(f"CREATE TABLE IF NOT EXISTS {nombreTabla} USING DELTA LOCATION '{pathSave}/'")

#### Acá se hace el join entre las predicciones y la base descriptiva:

In [0]:
BaseDescriptivaPwBi = spark.read.format("delta").load("dbfs:/mnt/gold/icbf-gai/BaseModeloProbabilisticoSabana/")
BasePrediccionesPwBi = spark.read.format("delta").load("dbfs:/mnt/gold/icbf-gai/BaseModeloProbabilisticoPredicciones/")
BaseDescriptivaPwBi.createOrReplaceTempView("BaseDescriptivaPwBi")
BasePrediccionesPwBi.createOrReplaceTempView("BasePrediccionesPwBi")

In [0]:
%sql
select * from BaseDescriptivaPwBi

In [0]:
BaseGrafica = spark.sql("""
SELECT
    BP.*,
    DB.FechaNacimientoBeneficiario,
    UPPER(DB.sexoBeneficiario) as sexoBeneficiario,
    UPPER(COALESCE(DB.NombreSexo,"No Definido")) AS NombreSexo,   
    UPPER(COALESCE(DB.RegimenSeguridadSocial,"No Definido")) AS RegimenSeguridadSocial,
    (YEAR(now()) - YEAR(DB.FechaNacimientoBeneficiario))  AS EdadBeneficiarioAnios, 
    ROUND(DATEDIFF(DATE(now()),DATE(DB.FechaNacimientobeneficiario))/30,0) as EdadBeneficiarioMeses,
    DB.CodPaisNacimientoBeneficiario,
    UPPER(COALESCE(DB.NombPaisNacimientoBeneficiario,"No Definido")) AS NombPaisNacimientoBeneficiario,   
    DB.CodPaisRasidenciaBen,
    UPPER(COALESCE(DB.NombPaisRasidenciaBen,"No Definido")) AS NombPaisRasidenciaBen,   
    CONCAT(DB.NombPaisRasidenciaBen,", ",DB.NomDepartamentoResidenciaBen,", ",DB.NomMunicipResidenciaBen,", Colombia") AS UbicGeogBen,
    DB.CodDepartamentoResidenciaBen,
    UPPER(COALESCE(DB.NomDepartamentoResidenciaBen,"No Definido")) AS NomDepartamentoResidenciaBen,   
    DB.CodMunicipResidenciaBen,
    UPPER(COALESCE(DB.NomMunicipResidenciaBen,"No Definido")) AS NomMunicipResidenciaBen,
    DB.CodComunaResidenciaBen,
    UPPER(COALESCE(DB.NomComunaResidenciaBen,"No Definido")) AS NomComunaResidenciaBen,
    DB.CodBarrioResidenciaBen,
    UPPER(COALESCE(DB.NomBarrioResidenciaBen,"No Definido")) AS NomBarrioResidenciaBen,
    DB.CodVeredaResidenciaBen,
    UPPER(COALESCE(DB.NomVeredaResidenciaBen,"No Definido")) AS NomVeredaResidenciaBen,
    DB.CodRancheriaResidenciaBen,
    UPPER(COALESCE(DB.NomRancheriaResidenciaBen,"No Definido")) AS NomRancheriaResidenciaBen,
    UPPER(COALESCE(DB.PresentaDiscapacidad,"No Definido")) AS PresentaDiscapacidad,
    UPPER(COALESCE(DB.ZonaUbicacionBeneficiario,"No Definido")) AS ZonaUbicacionBeneficiario,
    UPPER(COALESCE(DB.GrupoEtnico,"No Definido")) AS GrupoEtnico,
    DB.MesesLactanciaMaternaExclusiva,
    DB.MesesLactanciaMaternaTotal,
    DB.PesoBeneficiarioAlNacer,
    DB.TallaBeneficiarioAlNacer,
    UPPER(COALESCE(DB.AntecendentePremadurez,"No Definido")) AS AntecendentePremadurez,
    DB.EdadGestacionarAlNacer,
    left(now(),10) as FechaGeneracion    
FROM
    BasePrediccionesPwBi BP
    INNER JOIN BaseDescriptivaPwBi DB 
	ON BP.IdBeneficiario = DB.IdBeneficiario
""")
BaseGrafica.createOrReplaceTempView("BaseGrafica")

In [0]:
%sql
select * from BaseGrafica

In [0]:
#---------------------------- Borrar Histórico ----------------------------------------#
dbfsWriteDelta1 = "dbfs:/mnt/gold/icbf-gai/BaseModeloProbabilisticoGrafica/"
try:
    dbutils.fs.ls(dbfsWriteDelta1)
    print(" *** Borrando Directorio *** ")
    files = dbutils.fs.ls(dbfsWriteDelta1)
    for f in files:
        f.path
        dbutils.fs.rm(f.path, recurse=True)
except:
    print(" *** No Existe Directorio *** ")
    pass
#---------------------------- Directorio de Salida ------------------------------------#
pathSave = dbfsWriteDelta1
nombreTabla = "dbo.BaseGrafica"
BaseGrafica.write \
.format("delta") \
.mode("overwrite") \
.option("overwriteSchema", "true") \
.save(f"{pathSave}") 
print(" *** Guardado en el DataLake *** ")
spark.sql(f"CREATE TABLE IF NOT EXISTS {nombreTabla} USING DELTA LOCATION '{pathSave}/'")